In [5]:
import pandas as pd
# Load Base.csv
df = pd.read_csv('Variant.csv')

# Remove "device_fraud_count", it's 0 for all entries
print(df['device_fraud_count'].value_counts()) # It's 0 for all rows
df = df.drop(['device_fraud_count'], axis=1, errors='ignore')
df.shape
df.head()

device_fraud_count
0    60232
Name: count, dtype: int64


,fraud_bool,income,name_email_similarity,prev_address_months_count,current_address_months_count,customer_age,days_since_request,intended_balcon_amount,payment_type,zip_count_4w,...,proposed_credit_limit,foreign_request,source,session_length_in_minutes,device_os,keep_alive_session,device_distinct_emails_8w,month,x1,x2
0,0,0.2,0.369510,326,20,60,0.022212,-0.758101,AB,647,...,1000,0,INTERNET,19.450933,other,1,2,3,-0.282223,-0.137122
1,0,0.6,0.858929,-1,154,50,0.038221,-0.781645,AB,1238,...,1500,0,INTERNET,3.641099,windows,1,1,5,-0.145247,1.614553
2,0,0.9,0.166457,-1,41,50,0.002588,17.710636,AA,815,...,500,0,INTERNET,3.850864,macintosh,0,1,4,-0.029179,1.089166
3,0,0.8,0.250290,23,9,50,0.004026,18.224113,AA,1204,...,200,0,INTERNET,5.182197,windows,1,1,2,2.178365,1.301696
4,0,0.1,0.619087,-1,30,30,0.011687,-1.434154,AB,1182,...,500,0,INTERNET,4.346861,windows,1,1,7,1.472099,0.367675


In [6]:
'''用于将数据分为特征和目标变量。

首先，通过将'fraud_bool'列从DataFrame中删除，创建了一个名为X的新DataFrame，其中包含除了目标变量外的所有特征。

然后，创建了一个名为y的Series，其中只包含目标变量'fraud_bool'。

接下来，根据月份对数据进行了训练集和测试集的划分。月份0-5被划分为训练数据，而月份6-7被划分为测试数据。所以，X_train和y_train包含训练集数据，而X_test和y_test包含测试集数据。

最后，使用drop函数删除了X_train和X_test中的'month'列，因为该列已经不再需要作为特征变量。
'''

# Split data into features and target
X = df.drop(['fraud_bool'], axis=1)
y = df['fraud_bool']

# Train test split by 'month', month 0-5 are train, 6-7 are test data as proposed
X_train = X[X['month'] < 6]
X_test = X[X['month'] >= 6]
y_train = y[X['month'] < 6]
y_test = y[X['month'] >= 6]

X_train = X_train.drop('month', axis=1)
X_test = X_test.drop('month', axis=1)
X_train.head()

,income,name_email_similarity,prev_address_months_count,current_address_months_count,customer_age,days_since_request,intended_balcon_amount,payment_type,zip_count_4w,velocity_6h,...,has_other_cards,proposed_credit_limit,foreign_request,source,session_length_in_minutes,device_os,keep_alive_session,device_distinct_emails_8w,x1,x2
0,0.2,0.369510,326,20,60,0.022212,-0.758101,AB,647,5944.130884,...,0,1000,0,INTERNET,19.450933,other,1,2,-0.282223,-0.137122
1,0.6,0.858929,-1,154,50,0.038221,-0.781645,AB,1238,866.883575,...,0,1500,0,INTERNET,3.641099,windows,1,1,-0.145247,1.614553
2,0.9,0.166457,-1,41,50,0.002588,17.710636,AA,815,7784.667290,...,0,500,0,INTERNET,3.850864,macintosh,0,1,-0.029179,1.089166
3,0.8,0.250290,23,9,50,0.004026,18.224113,AA,1204,6510.930878,...,0,200,0,INTERNET,5.182197,windows,1,1,2.178365,1.301696
5,0.9,0.317856,83,6,50,0.024149,-1.323432,AC,477,5526.000890,...,0,1500,0,INTERNET,4.427453,other,0,1,0.330928,-1.505284


In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

"""首先，通过 X_train.dtypes == 'object' 判断每一列是否包含分类特征。这样得到一个布尔类型的 Series，其中索引是列名，值表示是否为分类特征。

接着，通过 list(s[s].index) 得到了所有包含分类特征的列的列表，并赋值给 object_cols。

然后，使用 OneHotEncoder 初始化一个独热编码器 ohe ，其中的参数 sparse=False 表示生成的编码结果不是稀疏矩阵，handle_unknown='ignore' 表示在测试集中出现未在训练集中出现的特征时忽略它。

接下来，分别用 ohe.fit_transform(X_train[object_cols]) 和 ohe.transform(X_test[object_cols]) 对训练集和测试集中的分类特征进行独热编码，分别得到 ohe_cols_train 和 ohe_cols_test，这里用 DataFrame 来存储编码结果。

再将 ohe_cols_train 和 ohe_cols_test 的索引设置为原始数据的索引，以便后续拼接回原始数据。

然后，从训练集和测试集中删除原始的分类特征列，得到只包含数值特征的 num_X_train 和 num_X_test。

最后，使用 pd.concat 将 num_X_train 和 ohe_cols_train 按列拼接，得到最终的训练集 X_train。同样地，将 num_X_test 和 ohe_cols_test 拼接为测试集 X_test。

为了确保新的特征列的名称与原始数据一致，通过 X_train.columns.astype(str) 将特征列的名字转换为字符串类型。"""


s = (X_train.dtypes == 'object') # list of column-names and wether they contain categorical features
object_cols = list(s[s].index) # All the columns containing these features
print(X[object_cols])

ohe = OneHotEncoder(sparse=False, handle_unknown='ignore') # ignore any features in the test set that were not present in the training set

# Get one-hot-encoded columns
ohe_cols_train = pd.DataFrame(ohe.fit_transform(X_train[object_cols]))
ohe_cols_test = pd.DataFrame(ohe.transform(X_test[object_cols]))

# Set the index of the transformed data to match the original data
ohe_cols_train.index = X_train.index
ohe_cols_test.index = X_test.index

# Remove the object columns from the training and test data
num_X_train = X_train.drop(object_cols, axis=1)
num_X_test = X_test.drop(object_cols, axis=1)

# Concatenate the numerical data with the transformed categorical data
X_train = pd.concat([num_X_train, ohe_cols_train], axis=1)
X_test = pd.concat([num_X_test, ohe_cols_test], axis=1)

# Newer versions of sklearn require the column names to be strings
X_train.columns = X_train.columns.astype(str)
X_test.columns = X_test.columns.astype(str)

# See that it replaced all categorical values
X_train.head(1)

      payment_type employment_status housing_status    source  device_os
0               AB                CC             BC  INTERNET      other
1               AB                CA             BB  INTERNET    windows
2               AA                CA             BC  INTERNET  macintosh
3               AA                CA             BC  INTERNET    windows
4               AB                CB             BB  INTERNET    windows
...            ...               ...            ...       ...        ...
60227           AD                CA             BB  INTERNET      other
60228           AA                CA             BC  INTERNET      linux
60229           AB                CA             BA  INTERNET    windows
60230           AB                CC             BC  INTERNET    windows
60231           AA                CA             BC  INTERNET      linux

[60232 rows x 5 columns]


C:\Users\admin\AppData\Roaming\Python\Python39\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,income,name_email_similarity,prev_address_months_count,current_address_months_count,customer_age,days_since_request,intended_balcon_amount,zip_count_4w,velocity_6h,velocity_24h,...,16,17,18,19,20,21,22,23,24,25
0,0.2,0.36951,326,20,60,0.022212,-0.758101,647,5944.130884,3862.31627,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [8]:
drop_cols = ['zip_count_4w', 'velocity_6h', 'velocity_24h', 'velocity_4w']

X_train.drop(columns=drop_cols, inplace=True)
X_test.drop(columns=drop_cols, inplace=True)

In [9]:
X_train.head()

,income,name_email_similarity,prev_address_months_count,current_address_months_count,customer_age,days_since_request,intended_balcon_amount,bank_branch_count_8w,date_of_birth_distinct_emails_4w,credit_risk_score,...,16,17,18,19,20,21,22,23,24,25
0,0.2,0.369510,326,20,60,0.022212,-0.758101,763,2,186,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.6,0.858929,-1,154,50,0.038221,-0.781645,929,6,197,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.9,0.166457,-1,41,50,0.002588,17.710636,1,7,212,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.8,0.250290,23,9,50,0.004026,18.224113,1,1,130,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
5,0.9,0.317856,83,6,50,0.024149,-1.323432,1,5,205,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [10]:
# Feature standardization
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [15]:
#Train and Test the model
from keras.layers import LSTM, Dense
from keras.models import Sequential

# Build LSTM Model
model = Sequential()
model.add(LSTM(64, return_sequences=True, input_shape=(X_train_scaled.shape[1], 1)))
model.add(LSTM(64))  # Stack Layer 2 LSTM
model.add(Dense(1, activation='sigmoid'))

#  Compilation Model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Model training
model.fit(X_train_scaled, y_train, epochs=10, batch_size=32)

# Make predictions on the test set
y_pred = model.predict(X_test_scaled)


Epoch 1/10
1427/1427 [==============================] - 36s 23ms/step - loss: 0.0677 - accuracy: 0.9883
Epoch 2/10
1427/1427 [==============================] - 34s 24ms/step - loss: 0.0594 - accuracy: 0.9885
Epoch 3/10
1427/1427 [==============================] - 34s 24ms/step - loss: 0.0511 - accuracy: 0.9887
Epoch 4/10
1427/1427 [==============================] - 34s 24ms/step - loss: 0.0476 - accuracy: 0.9894
Epoch 5/10
1427/1427 [==============================] - 34s 24ms/step - loss: 0.0463 - accuracy: 0.9897
Epoch 6/10
1427/1427 [==============================] - 33s 23ms/step - loss: 0.0451 - accuracy: 0.9901
Epoch 7/10
1427/1427 [==============================] - 34s 23ms/step - loss: 0.0440 - accuracy: 0.9905
Epoch 8/10
1427/1427 [==============================] - 33s 23ms/step - loss: 0.0433 - accuracy: 0.9907
Epoch 9/10
1427/1427 [==============================] - 33s 23ms/step - loss: 0.0429 - accuracy: 0.9906
Epoch 10/10
456/456 [==============================] - 5s 9ms/st

In [16]:
from sklearn.metrics import accuracy_score

# Converts probabilities to category labels
threshold = 0.5  # set threshold
y_pred_labels = [1 if prob >= threshold else 0 for prob in y_pred]

# Calculation accuracy
accuracy = accuracy_score(y_test, y_pred_labels)

# Print accuracy
print("Accuracy: {:.2f}%".format(accuracy * 100))


Accuracy: 99.02%
